In [1]:
import functools

import jax
import numpy as np
import scanpy as sc

from cfp.metrics import compute_mean_metrics, compute_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp

In [2]:
split = 2

In [3]:
adata_pred_ood = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/models/otfm/combosciplex/adata_ood_with_predictions_{split}_mean_pooling.h5ad")
adata_pred_test = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/models/otfm/combosciplex/adata_test_with_predictions_{split}_mean_pooling.h5ad")

In [4]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [5]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [6]:
adata_ref_test = adata_test[adata_test.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_test, n_comps=10)

adata_ref_ood = adata_ood[adata_ood.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_ood, n_comps=10)

In [7]:
cfpp.project_pca(query_adata=adata_pred_test, ref_adata=adata_ref_test, layer="X_recon_pred")
cfpp.project_pca(query_adata=adata_test, ref_adata=adata_ref_test)
test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if cond == "control":
        continue
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm["X_pca"]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X.toarray()
    test_data_target_decoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].layers["X_recon_pred"]
    test_data_target_encoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["X_pca"]

cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_ref_ood, layer="X_recon_pred")
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_ref_ood)
ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm["X_pca"]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.toarray()
    ood_data_target_decoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].layers["X_recon_pred"]
    ood_data_target_encoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["X_pca"]

In [8]:
test_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in test_data_target_decoded_predicted.keys()
}
ood_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [9]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)

test_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, test_data_target_decoded_predicted, test_deg_dict)
test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded, test_deg_dict)


In [10]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")

deg_test_metrics = jax.tree_util.tree_map(compute_metrics, test_deg_target_decoded, test_deg_target_decoded_predicted)
deg_mean_test_metrics = compute_mean_metrics(deg_test_metrics, prefix="deg_test_")


In [11]:
test_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, test_data_target_encoded, test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

test_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics_fast, test_data_target_decoded, test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [12]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [13]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.9581059073409444,
 'decoded_ood_sinkhorn_div_1': 120.8655253819057,
 'decoded_ood_sinkhorn_div_10': 72.63797869001117,
 'decoded_ood_sinkhorn_div_100': 4.478858947753906,
 'decoded_ood_e_distance': 8.107913985555339,
 'decoded_ood_mmd': 0.01375586140368666}

In [14]:
mean_ood_metrics_encoded

{'encoded_ood_r_squared': 0.6516077670571899,
 'encoded_ood_sinkhorn_div_1': 6.189871038709368,
 'encoded_ood_sinkhorn_div_10': 4.13825375693185,
 'encoded_ood_sinkhorn_div_100': 3.44680050441197,
 'encoded_ood_e_distance': 6.732766303517424,
 'encoded_ood_mmd': 0.043304363930863995}

In [15]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.9510656265844923,
 'deg_ood_sinkhorn_div_1': 14.521922520228795,
 'deg_ood_sinkhorn_div_10': 2.255475180489676,
 'deg_ood_sinkhorn_div_100': 1.661040987287249,
 'deg_ood_e_distance': 3.21189171305886,
 'deg_ood_mmd': 0.01580686561231102}

In [16]:
deg_mean_test_metrics

{'deg_test_r_squared': 0.9906559703690511,
 'deg_test_sinkhorn_div_1': 12.0661248366038,
 'deg_test_sinkhorn_div_10': 1.1918528874715169,
 'deg_test_sinkhorn_div_100': 0.32238515218098956,
 'deg_test_e_distance': 0.5366546295596041,
 'deg_test_mmd': 0.012183367701557776}

In [17]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.9863166354303625,
 'decoded_test_e_distance': 2.546032263460942,
 'decoded_test_mmd_distance': 0.015391907887533307}

In [18]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.9581059073409444,
 'decoded_ood_sinkhorn_div_1': 120.8655253819057,
 'decoded_ood_sinkhorn_div_10': 72.63797869001117,
 'decoded_ood_sinkhorn_div_100': 4.478858947753906,
 'decoded_ood_e_distance': 8.107913985555339,
 'decoded_ood_mmd': 0.01375586140368666}

In [19]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.9510656265844923,
 'deg_ood_sinkhorn_div_1': 14.521922520228795,
 'deg_ood_sinkhorn_div_10': 2.255475180489676,
 'deg_ood_sinkhorn_div_100': 1.661040987287249,
 'deg_ood_e_distance': 3.21189171305886,
 'deg_ood_mmd': 0.01580686561231102}

In [20]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.9863166354303625,
 'decoded_test_e_distance': 2.546032263460942,
 'decoded_test_mmd_distance': 0.015391907887533307}

In [21]:
mean_test_metrics_encoded

{'encoded_test_r_squared': 0.9812498472872586,
 'encoded_test_sinkhorn_div_1': 3.73487796386083,
 'encoded_test_sinkhorn_div_10': 0.9835970799128214,
 'encoded_test_sinkhorn_div_100': 0.31789565086364746,
 'encoded_test_e_distance': 0.4941081958143024,
 'encoded_test_mmd': 0.015069589950144291}

In [22]:
ood_metrics_encoded

{'Dacinostat+Dasatinib': {'r_squared': 0.10585335518440553,
  'sinkhorn_div_1': 6.438941955566406,
  'sinkhorn_div_10': 4.3116302490234375,
  'sinkhorn_div_100': 3.7880430221557617,
  'e_distance': 7.4753081221563225,
  'mmd': 0.05123407},
 'Dacinostat+PCI-34051': {'r_squared': 0.2505135735379732,
  'sinkhorn_div_1': 5.862081527709961,
  'sinkhorn_div_10': 3.8926868438720703,
  'sinkhorn_div_100': 3.395838737487793,
  'e_distance': 6.689210314780867,
  'mmd': 0.047959387},
 'Givinostat+Cediranib': {'r_squared': 0.9918570934422293,
  'sinkhorn_div_1': 1.3619508743286133,
  'sinkhorn_div_10': 0.3136463165283203,
  'sinkhorn_div_100': 0.1293048858642578,
  'e_distance': 0.2289501500129596,
  'mmd': 0.006049788},
 'Givinostat+Curcumin': {'r_squared': 0.9810752638357901,
  'sinkhorn_div_1': 1.4867157936096191,
  'sinkhorn_div_10': 0.4879112243652344,
  'sinkhorn_div_100': 0.2953987121582031,
  'e_distance': 0.5435285544818171,
  'mmd': 0.006319406},
 'Panobinostat+Alvespimycin': {'r_squared

In [23]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/results/otfm"

In [24]:
import os
import pandas as pd

pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, f"ood_metrics_encoded_{split}_mean_pooling.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, f"ood_metrics_decoded_{split}_mean_pooling.csv"))
pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, f"test_metrics_encoded_{split}_mean_pooling.csv"))
pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, f"test_metrics_decoded_{split}_mean_pooling.csv"))
pd.DataFrame.from_dict(deg_test_metrics).to_csv(os.path.join(output_dir, f"test_metrics_deg_{split}_mean_pooling.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, f"ood_metrics_ood_{split}_mean_pooling.csv"))
